# Automated ML
TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.automl
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset
from azureml.train.automl.utilities import get_primary_metrics
from azureml.train.automl import AutoMLConfig
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.automl.core.shared import constants


import os
import json
import joblib
import logging
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

## Initializing a Workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-190098
aml-quickstarts-190098
southcentralus
1b944a9b-fdae-4f97-aeb1-b7eea0beac53


TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

## Creating an AML Experiment

In [3]:
# choose a name for experiment
experiment_name = 'automl-heart-failure'
experiment = Experiment(ws, experiment_name)
run = experiment.start_logging()

experiment

Name,Workspace,Report Page,Docs Page
automl-heart-failure,quick-starts-ws-190098,Link to Azure Machine Learning studio,Link to Documentation


## Creating a Compute Cluster

In [4]:
amlcompute_cluster_name = "hf-compute"

try:
    compute_target = ComputeTarget(workspace = ws, name = amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_DS3_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded................................................................................................................

KeyboardInterrupt: 

## Data Wrangling

## Data Gathering

### Overview
In this project, the Heart Failure Prediction dataset from Kaggle is used. The description of the dataset is provided below.

**The dataset has the following features:**  
* age: Age (numeric)
* anaemia: Decrease of red blood cells or hemoglobin (boolean)
* creatinine_phosphokinase: Level of the CPK enzyme in the blood (mcg/L)
* diabetes: If the patient has diabetes (boolean)
* ejection_fraction: Percentage of blood leaving the heart at each contraction (percentage)
* high_blood_pressure: If the patient has hypertension (boolean)
* platelets: Platelets in the blood (kiloplatelets/mL)
* serum_creatinine: Level of serum creatinine in the blood (mg/dL)
* serum_sodium: Level of serum sodium in the blood (mEq/L)
* sex: Woman or man (binary)

**The task:**  
Developing a ML model to predict death events using 12 clinical features.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
### Gathering the Dataset
data_path = "https://raw.githubusercontent.com/ugururesin/Azure-ML-Capstone-Project/main/dataset/heart_failure_clinical_records_dataset.csv"


exist = False
key = "heart-failure"
description_text = "heart failure dataset for binary classification"

if key in ws.datasets.keys(): 
        exist = True
        dataset = ws.datasets[key] 
        

if not exist:
        #Creating automl dataset and registering it into the workspace
        dataset = Dataset.Tabular.from_delimited_files(data_path)        
        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

## Data Assesing

In [ ]:
df = dataset.to_pandas_dataframe()
df.head()

In [ ]:
#Creating training and testing datasets
x = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

train_df = pd.concat([x_train, y_train], axis=1)
test_df = pd.concat([x_test, y_test], axis=1)

train_df.head()

In [ ]:
#Since pandas dataframe is not supported, the dataset needs to be converted to tabular dataset
if not os.path.isdir('dataset'):
    os.mkdir('dataset')

pd.DataFrame(train_df).to_csv("dataset/train_data.csv", index=False)
pd.DataFrame(test_df).to_csv("dataset/test_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir = './dataset', target_path = 'heart-failure-dataset/train_data.csv')

train_data = Dataset.Tabular.from_delimited_files(path = ds.path('heart-failure-dataset/train_data.csv'))

In [ ]:
train_data

### Checking the target feature

In [ ]:
df['DEATH_EVENT'].value_counts()

In [ ]:
get_primary_metrics("classification")

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

* **experiment_timeout_minutes**: 15 minutes is set since the dataset is small.

* **iterations**: 50 is set since the dataset is small.  

* **max_concurrent_iterations**: Represents the maximum number of iterations that would be executed in parallel.

* **n_cross_validations**: To avoid overfitting and ensure randomness.

* **primary_metric**: Accuracy is selected to be able to compare with HyperDrive's results.

* **task**: Classification is selected since the response variable is binary (0 or 1).  

* **enable_early_stopping**: Activated to avoid overfitting.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,
    "iterations": 50,
    "max_concurrent_iterations": 3,
    "n_cross_validations": 3,
    "primary_metric" : 'accuracy',
    "featurization" : 'auto',
    "verbosity": logging.INFO
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task = "classification", 
    training_data = train_data,
    label_column_name="DEATH_EVENT",
    enable_early_stopping= True,
    debug_log = "automl_errors.log",
    compute_target = compute_target,
    **automl_settings
)
                             

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)
remote_run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_metrics = best_run.get_metrics()

print("Best run id : {}".format(best_run.id), 
      "AUC : {}".format(best_metrics['accuracy']), 
      "Best metrics : {}".format(best_metrics), 
      "Best model : {}".format(fitted_model), sep = '\n\n')

In [ ]:
#the fitted model
_, fitted_model = remote_run.get_output()
print(fitted_model)

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name = "heart-failure-best-model-automl", model_path = './outputs/model.pkl')
print(model)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# downloading score.py and env file
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# registering the model
model_name = best_run.properties['model_name']
description = 'AutoML model trained on heart failure data so as to predict whether death event occurs based on the clinical values'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id) # This will be written to the script file later in the notebook.

In [ ]:
model_name = 'best_run_automl.pkl'
model = remote_run.register_model(model_name = model_name)
environment = best_run.get_environment()

entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)

#configuration for inference
inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

#configuration for deployment and enabling the application insights
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

#deploying the model using azure container instance
service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

### Consume the Model

In [ ]:
%run endpoint.py

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
webservice.get_logs()

In [ ]:
#webservice.delete()
#compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
